In [13]:
import pandas as pd
import numpy as np

In [14]:
data=pd.read_csv("data/gemstone.csv")

In [15]:
data.drop(labels=["id"],axis=1,inplace=True)

In [16]:
# # it is a oridinal encoding
# cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
# clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}
# color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}


In [17]:
# data["cut"]=data["cut"].map(cut_map)
# data["clarity"]=data["clarity"].map(clarity_map)
# data["color"]=data["color"].map(color_map)

In [18]:
data

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [19]:
X = data.drop(labels=['price'],axis=1)

In [20]:
y = data[['price']]

In [21]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [27]:
categorical_columns=X.select_dtypes(include='object').columns

In [28]:
numerical_columns=X.select_dtypes(exclude='object').columns

In [29]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [30]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [33]:
num_pipline  = Pipeline(
    [
        ('Missing_value_handeler', SimpleImputer()),
        ("Standerd_scler",StandardScaler())
    ]
)

In [34]:
cat_pipline = Pipeline(
    [
        ('Imputer', SimpleImputer(strategy='most_frequent')),
        ('Ordinal_encoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

In [37]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipline,numerical_columns),
        ('cat_pipeline',cat_pipline,categorical_columns)
    ]
)

In [38]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [39]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [40]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [41]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [44]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [45]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [46]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344002
MAE: 674.025511579685
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8723151049944
MAE: 675.2036880701621
R2 score 93.68948574778251


Ridge
Model Training Performance
RMSE: 1013.9058997761052
MAE: 674.0565132295944
R2 score 93.6890676674133


Elasticnet
Model Training Performance
RMSE: 1513.914035022976
MAE: 1050.7468664314322
R2 score 85.92978759337907


